In [1]:
#Imports
import os
import tensorflow as tf

import sys
sys.path.insert(0, '..')
from utils.federatedAggregation import FederatedAggregation
from utils.reinforcementLearningHelper import *
from utils.federatedLearningHelper import *

In [8]:
#Setup Environments of selected buildings for training, evaluation, and testing

environments, observation_spec, action_spec, data  = setup_energymanagement_environments2(num_buildings=30, ecoPriority=0, noise_scale=0.8, return_dataset=True)

#Check environment setup
print(
    "Batch size:", environments["train"][f"building_1"].batch_size, 
    "/ State Space: {} / Action Space: {}".format(observation_spec.shape[0], action_spec.shape[0]),
    "/ Upper bound: {}".format(round(environments["train"][f"building_1"].action_spec().maximum.item(), 3)),
)

Batch size: 1 / State Space: 58 / Action Space: 1 / Upper bound: 2.3


In [11]:
data["train"]["building_1"]

,load_1,pv_1,price,emissions
0,1.553,0.0,0.21846,1.066200
1,1.715,0.0,0.21836,1.064706
2,1.339,0.0,0.22172,1.061807
3,0.865,0.0,0.22099,1.064542
4,0.380,0.0,0.21798,1.062929
...,...,...,...,...
17515,0.228,0.0,0.21440,1.086608
17516,0.458,0.0,0.21464,1.093526
17517,0.685,0.0,0.21530,1.079257
17518,0.650,0.0,0.21998,1.080775


In [3]:
# Cluster similar buildings (K-Means with DTW)

#Set parameter
num_clusters = 9

clustered_buildings = prosumption_clustered_buildings(num_clusters)
clustered_buildings

{0: array([7], dtype=int64),
 1: array([ 3,  4,  5,  9, 10, 11, 16, 23, 25, 26, 27, 30], dtype=int64),
 2: array([1], dtype=int64),
 3: array([24], dtype=int64),
 4: array([12, 13, 14, 15, 19, 20, 22, 28, 29], dtype=int64),
 5: array([ 6, 17], dtype=int64),
 6: array([8], dtype=int64),
 7: array([ 2, 18], dtype=int64),
 8: array([21], dtype=int64)}

In [4]:
# Setup Agent networks
federated_rounds = 3
batch_size = 128
replay_buffer_capacity = 20000 #-> only <18.000 samples per dataset
initial_collect_steps = 2000 #2000
collect_steps_per_iteration = 30 
num_iterations = 5000 #10000
eval_interval = num_iterations-1

In [5]:
csv_name = f"NEW_FL_DDPG_OnlyCosts_FORECAST08"

In [6]:
# FEDERATED LEARNING - Initalization Round 0

tf.compat.v1.reset_default_graph()
global_step = tf.compat.v1.train.get_or_create_global_step()

#Initalize a global model for each Cluster of similar buildings
for cluster in range(num_clusters):
        
        # 1. Build global agent per cluster
        global_ddpg_agent, global_eval_policy, global_collect_policy = initialize_ddpg_agent(
                observation_spec=observation_spec, action_spec=action_spec, global_step=global_step, environments=environments,
                )

        # 2. Initially store weights
        model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster}/FLround{0}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        
        save_ddpg_weights(global_ddpg_agent, model_dir)

In [7]:
#FEDERATED LEARNING - Model training for multiple Rounds

#For each federated round and cluster
for federated_round  in range(federated_rounds):
    for cluster_number, buildings_in_cluster in clustered_buildings.items():

        #Iterate through the buildings per cluster
        print(f"Cluster {cluster_number}: Buildings {buildings_in_cluster} Federated round ---", federated_round+1, f"/ {federated_rounds}")
        local_storage = {
            "actor_weights": [], "critic_weights": [], "target_actor_weights": [], "target_critic_weights": [],"performance_metrics": []
            }
        
        for building_index in buildings_in_cluster:
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()
            
            #1. Initalize local agent
            local_ddpg_agent, local_eval_policy, local_collect_policy = initialize_ddpg_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster_number}/FLround{federated_round}_c{num_clusters}_AvgAgg")
            local_ddpg_agent = set_weights_to_ddpg_agent(local_ddpg_agent, model_dir)
            
            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            local_iterator, local_collect_driver, local_time_step, local_policy_state = setup_rl_training_pipeline(
                local_ddpg_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                local_collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Train, evaluate agent and store weights
            local_ddpg_agent, local_storage = local_agent_training_and_evaluation(
                local_iterator, local_collect_driver, local_time_step, local_policy_state, global_step, 
                local_ddpg_agent, local_eval_policy, local_storage, building_index, num_iterations, environments, agent_type="ddpg"
                )           

        # Performe Federated aggregation
        average_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["actor_weights"], local_storage["performance_metrics"])
        average_critic_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["critic_weights"], local_storage["performance_metrics"]) 
        average_target_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["target_actor_weights"], local_storage["performance_metrics"]) 
        average_target_critic_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["target_critic_weights"], local_storage["performance_metrics"])    
        #average_actor_weights = FederatedAggregation.federated_average_aggregation(local_storage["actor_weights"])
        #average_critic_weights = FederatedAggregation.federated_average_aggregation(local_storage["critic_weights"]) 
        #average_target_actor_weights = FederatedAggregation.federated_average_aggregation(local_storage["target_actor_weights"]) 
        #average_target_critic_weights = FederatedAggregation.federated_average_aggregation(local_storage["target_critic_weights"])    
        
        #Save federated weights for next round (Round + 1)
        model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster_number}/FLround{federated_round+1}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        save_federated_ddpg_weights(model_dir, average_actor_weights, average_critic_weights, average_target_actor_weights, average_target_critic_weights)

Cluster 0: Buildings [7] Federated round --- 1 / 3
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Return:  -3985.6711
Cluster 1: Buildings [ 3  4  5  9 10 11 16 23 25 26 27 30] Federated round --- 1 / 3
Return:  -2687.3389
Return:  -4638.022
Return:  -3264.9624
Return:  -4308.3794
Return:  -6406.356
Return:  -4972.1743
Return:  -8029.235
Return:  -4582.113
Return:  -5211.392
Return:  -7086.44
Return:  -5563.5044
Return:  -3043.8813
Cluster 2: Buildings [1] Federated round --- 1 / 3
Return:  -4227.8975
Cluster 3: Buildings [24] Federated round --- 1 / 3
Return:  -1358.6073
Cluster 4: Buildings [12 13 14 15 19 20 22 28 29] Federated round --- 1 / 3
Return:  -5065.42
Return:  -3050.139
Return:  -2246.4678
Return:  -2653.4282
Return:  -1586.3353
Return:  -3414.2686
Return:  -3716.1897
Return:  -3154.5195
Return:  -4174.063
Cluster 5: Buildings [ 6 17] Federated round --- 1 / 3
Return:  -9673.345
Return:  -7144.2715
Cluster 6: Buildings [8] Federat

In [8]:
# LOCAL REFITTING AND EVALUATION

best_federated_round = 3
num_rounds=3
num_test_iterations = 1

result_df = pd.DataFrame(columns=['Building', 'Total Profit'])

for cluster_number, buildings_in_cluster in clustered_buildings.items():
    for building_index in buildings_in_cluster:
        
        for round in range(num_rounds):
            print("Cluster: ", cluster_number, " - Building: ", building_index, " - round: ", round)
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()

            #1. Initalize local agent
            tf_ddpg_agent, eval_policy, collect_policy = initialize_ddpg_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster_number}/FLround{best_federated_round}_c{num_clusters}_AvgAgg")
            tf_agent = set_weights_to_ddpg_agent(tf_ddpg_agent, model_dir)

            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            iterator, collect_driver, time_step, policy_state = setup_rl_training_pipeline(
                tf_ddpg_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Setup wandb logging
            artifact = initialize_wandb_logging(name=f"{csv_name}_Home{building_index}_rd{round}", num_iterations=num_iterations)
            
            #5. Train, evaluate agent and store weights
            result_df, metrics = agent_training_and_evaluation(global_step, num_test_iterations, collect_driver, 
                time_step, policy_state, iterator, tf_ddpg_agent, eval_policy, building_index, result_df, eval_interval, environments)
            
            #6. End and log wandb
            end_and_log_wandb(metrics, artifact)

Cluster:  0  - Building:  7  - round:  0


c:\Users\rs1044\Documents\GitHub\Federated-Reinforcement-Learning-for-Battery-Charging\src\notebooks\..\utils\reinforcementLearningHelper.py:443: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat(


Building:  7  - Total Profit:  -645.859116323625  - Total Emissions:  2987.374196997361
Cluster:  0  - Building:  7  - round:  1
Building:  7  - Total Profit:  -644.0010740081161  - Total Emissions:  2981.0468143088874
Cluster:  0  - Building:  7  - round:  2
Building:  7  - Total Profit:  -635.1418932293865  - Total Emissions:  2905.3560513716125
Cluster:  1  - Building:  3  - round:  0
Building:  3  - Total Profit:  -275.08488717080826  - Total Emissions:  1227.0653055097205
Cluster:  1  - Building:  3  - round:  1
Building:  3  - Total Profit:  -275.75054029908716  - Total Emissions:  1231.9918487631498
Cluster:  1  - Building:  3  - round:  2
Building:  3  - Total Profit:  -275.4613973501881  - Total Emissions:  1229.6236551492264
Cluster:  1  - Building:  4  - round:  0
Building:  4  - Total Profit:  -923.5768260595789  - Total Emissions:  3689.6231309698737
Cluster:  1  - Building:  4  - round:  1
Building:  4  - Total Profit:  -922.9944737926972  - Total Emissions:  3683.6146939

In [9]:
# Save results
result_df['Setup'] = csv_name
result_df.index.name = 'Building_nr'
result_df.reset_index(inplace=True, drop=True)
os.makedirs('results', exist_ok=True)
result_df.to_csv(f'results/{csv_name}.csv', index=False)

print("Final result: ", result_df["Total Profit"].sum())

Final result:  -98925.25960224598


In [10]:
import requests

def send_telegram_message(bot_token, chat_id, message):
    """Send a message to a Telegram chat via the Bot API."""
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"
    }
    response = requests.post(url, json=payload)
    return response.json()

# Use the function
bot_token = os.getenv('TELEGRAM_BOT_TOKEN')
chat_id = os.getenv('TELEGRAM_CHAT_ID')
message = f"Script PC Home ist fertig!"

result = send_telegram_message(bot_token, chat_id, message)
print(result)

{'ok': True, 'result': {'message_id': 274, 'from': {'id': 7071194232, 'is_bot': True, 'first_name': 'Reinforcement Learning', 'username': 'FederatedRL_Bot'}, 'chat': {'id': 5493937056, 'first_name': 'Jonas', 'last_name': 'Sievers', 'username': 'JonasSievers', 'type': 'private'}, 'date': 1722560776, 'text': 'Script PC Home ist fertig!'}}


In [13]:
result_df['Total Profit'].sum()

-98925.25960224598

In [14]:
std_dev = result_df.groupby('Building')['Total Profit'].std().mean()
std_dev

1.3223087611734288